In [1]:
#BSD 3-Clause License
#
#Copyright (c) 2025, ASU-VDA-Lab
#
#Redistribution and use in source and binary forms, with or without
#modification, are permitted provided that the following conditions are met:
#
#1. Redistributions of source code must retain the above copyright notice, this
#   list of conditions and the following disclaimer.
#
#2. Redistributions in binary form must reproduce the above copyright notice,
#   this list of conditions and the following disclaimer in the documentation
#   and/or other materials provided with the distribution.
#
#3. Neither the name of the copyright holder nor the names of its
#   contributors may be used to endorse or promote products derived from
#   this software without specific prior written permission.
#
#THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
#AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
#IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
#DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
#FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
#DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
#SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
#CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
#OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
#OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

import sys
sys.path.append("./")
from src import VAE, StableDiffusionPipeline, EDAUNet
import torch
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
from diffusers import DDPMScheduler
import matplotlib.pyplot as plt

# Define model path here

In [2]:
vae_path = "./models/vae/vae.pt"
unet_path = "./models/unet/unet.pt"
device ="cuda:0"

# Load models and launch the pipeline

In [3]:
# Load VAE
vae = VAE()
vae.load_state_dict(torch.load(vae_path))
vae = vae.to(dtype = torch.float32, device = device)
# Load UNet
unet = EDAUNet()
unet.load_state_dict(torch.load(unet_path, map_location = device), strict = False)
# Load scheduler
scheduler = DDPMScheduler.from_pretrained("stabilityai/stable-diffusion-2-1", subfolder = "scheduler")
# Launch pipeline
pipeline = StableDiffusionPipeline(vae = vae, unet = unet, scheduler = scheduler)
pipeline = pipeline.to(device = device)

# Set circuit information

In [6]:
height = 256 # Works best for 1:1 aspect ratio
width = 256 
macros = 3 # Setting this too large might result in failed sampling of macro bounding boxes
utilization = 0.9 # Recommend to set this from 0.65 to 0.9
clock_period = 20.0 # Recommend to set this from 2.0 to 25.0

# Generate synthetic circuit heatmaps

In [ ]:

images, iterations = pipeline(
  height = height,
  width = width,
  macros = macros,
  utilization = utilization,
  clock_period = clock_period,
  num_inference_steps = 100,
  guidance_scale = 1.0,
  num_images_per_prompt = 1,
  output_type = "numpy",
)

# Show iterations used to generate the circuit heatmaps

In [ ]:
iterations

# Show the heatmaps

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(15, 10))

im00 = axs[0, 0].imshow(images["cell_density"][0], vmin=0, vmax=1, origin='lower', cmap = 'jet')
axs[0, 0].set_title("Cell Density")
fig.colorbar(im00, ax=axs[0, 0])

im10 = axs[1, 0].imshow(images["macro_region"][0], vmin=0, vmax=1, origin='lower', cmap = 'jet')
axs[1, 0].set_title("Macro Region")
fig.colorbar(im10, ax=axs[1, 0])

im01 = axs[0, 1].imshow(images["RUDY"][0], vmin=0, vmax=1, origin='lower', cmap = 'jet')
axs[0, 1].set_title("RUDY")
fig.colorbar(im01, ax=axs[0, 1])

im11 = axs[1, 1].imshow(images["IR_drop"][0], vmin=0, vmax=1, origin='lower', cmap = 'jet')
axs[1, 1].set_title("IR Drop")
fig.colorbar(im11, ax=axs[1, 1])

im02 = axs[0, 2].imshow(images["power_all"][0], vmin=0, vmax=1, origin='lower', cmap = 'jet')
axs[0, 2].set_title("Power All")
fig.colorbar(im02, ax=axs[0, 2])

im12 = axs[1, 2].imshow(images["power_sca"][0], vmin=0, vmax=1, origin='lower', cmap = 'jet')
axs[1, 2].set_title("Power SCA")
fig.colorbar(im12, ax=axs[1, 2])

plt.tight_layout()
plt.show()